In [1]:
cd "/suphys/aloe8475"


/suphys/aloe8475


In [2]:
cd "Documents/edamame"

/import/silo2/aloe8475/Documents/edamame


In [3]:
from scipy.io import loadmat, savemat
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import datetime
import networkx as nx
from edamame import *
from tqdm import tqdm_notebook
import os
import edamame.core.wires as wires
from random import choice
import warnings
from IPython.core.debugger import set_trace
import nct
import bct

#warnings.filterwarnings('ignore')

In [4]:
import pickle 
import _pickle as cPickle
import gzip
def compressed_pickle(obj, filename,protocol=-1):
    with gzip.open(filename, 'wb') as f:
        cPickle.dump(obj, f, protocol)

In [5]:
# this_seed2=700
def connected_component_subgraphs(G):
    for c in nx.connected_components(G):
        yield G.subgraph(c)

In [6]:
#Select Largest Components
def select_largest_component_new(wires_dict):
    """
    Find and select largest connected component of the original graph G.
    Throws away unconnected components and updates all the keys in wires_dict 
    """
#     def connected_component_subgraphs(G):
#         for c in nx.connected_components(G):
#             yield G.subgraph(c)
    
    wires_dict['G'] = max(connected_component_subgraphs(wires_dict['G']), key=len)
#     set_trace()
    nw = len(wires_dict['G'].nodes())
    nj = len(wires_dict['G'].edges())   
    
    logging.info("The largest component has %5d nodes and %6d edges", nw, nj)

    # Replace values in the dictionary
    wires_dict['number_of_wires']     = nw
    wires_dict['number_of_junctions'] = nj
    wires_dict['xa'] = wires_dict['xa'][wires_dict['G'].nodes()] 
    wires_dict['ya'] = wires_dict['ya'][wires_dict['G'].nodes()] 
    wires_dict['xb'] = wires_dict['xb'][wires_dict['G'].nodes()] 
    wires_dict['yb'] = wires_dict['yb'][wires_dict['G'].nodes()]
    wires_dict['xc'] = wires_dict['xc'][wires_dict['G'].nodes()] 
    wires_dict['yc'] = wires_dict['yc'][wires_dict['G'].nodes()]
 
    # Keep old edge_list
    old_edge_list = [(ii, kk) for ii, kk in  zip(wires_dict['edge_list'][:, 0], wires_dict['edge_list'][:, 1])]
    # Remove old edge list
    wires_dict = wires.remove_key(wires_dict, 'edge_list') 
    # Save indices of intersections in the old graph
    ind_dict = {key:value for value,key in enumerate(old_edge_list)}
    new_edge_list = sorted([kk if kk[0] < kk[1] else (kk[1], kk[0]) for kk in wires_dict['G'].edges()], key=lambda x: x[0])
    # Find intersection between the two sets
    inter = set(ind_dict).intersection(new_edge_list)
    # Retrieve edge indices/positions from the old list
    edges_idx = [ind_dict[idx] for idx in inter]
       
    # These have length equal to number of junctions -- only get the ones we need
    wires_dict['xi'] = wires_dict['xi'][edges_idx] 
    wires_dict['yi'] = wires_dict['yi'][edges_idx] 
    
    # Get contiguous numbering of nodes
    # Build node mapping 
    node_mapping    = {key:value for value, key in enumerate(sorted(wires_dict['G'].nodes()))}
    # This  step also renames edges list
    wires_dict['G'] =  nx.relabel_nodes(wires_dict['G'] , node_mapping)

    # Swap node vertices if vertex 0 is larger than vertex 1, then sort by first element
    wires_dict['edge_list'] = np.asarray(sorted([kk if kk[0] < kk[1] else (kk[1], kk[0]) for kk in wires_dict['G'].edges()], key=lambda x: x[0]))
    
    # Save adjacency matrix of new graph
    wires_dict = wires.remove_key(wires_dict, 'adj_matrix') 
    wires_dict = wires.generate_adj_matrix(wires_dict)

    wire_distances = wires.cdist(np.array([wires_dict['xc'], wires_dict['yc']]).T, np.array([wires_dict['xc'], wires_dict['yc']]).T, metric='euclidean')    
    wires_dict['wire_distances'] = wire_distances

    return wires_dict 

In [7]:
cd "/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks/"

/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks


In [8]:
data=loadmat('avgDegreeVals_allASN.mat')
avgDeg=data['avgdeg'][0]

numNodes=data['numNodes'][0]

In [9]:
ResultSqu_Grid=[None]*len(avgDeg)
ResultSqu_Random=[None]*len(avgDeg)

In [10]:
# seed=[]
# for i in range(len(avgDeg)):
#     seed.append(np.random.randint(10000))

In [11]:
#get seed from previous networks:
name='WS_Allnetworks_LinearTransformation.pkl'
file = open(name, 'rb')
#     [ASN300,cluster1,cluster2,cluster3,time_index,nodesList] = pickle.load(file)
#     [ASN300,cluster1,cluster2,cluster3] = pickle.load(file)
[WS_Grid,WS_Random] = pickle.load(file)
seed=[]
for i in range(len(WS_Random)):
    seed.append(WS_Random[i]['seed'])

In [12]:
#create watts strogatz graphs:
random=[]
grid=[]
temp=[]
temp2=[]
for i in range(len(avgDeg)):
    k=int(np.round(avgDeg[i]))
    temp2.append(nx.watts_strogatz_graph(numNodes[i],2*k,1,seed=seed[i]))
    temp.append(nx.watts_strogatz_graph(numNodes[i],2*k,0,seed=seed[i]))
    grid.append(max(connected_component_subgraphs(temp[i]), key=len))
    random.append(max(connected_component_subgraphs(temp2[i]), key=len))

In [13]:
temp_rand=[]
temp=[]
shortestPath_Grid=[]
shortestPath_Random=[]
onAmp_Grid=[]
onAmp_Random=[]

for i in tqdm(range(len(avgDeg))):
    temp_rand.append(getFarthestPairing(nx.adj_matrix(random[i]).todense()))
    temp.append(getFarthestPairing(nx.adj_matrix(grid[i]).todense()))
    shortestPath_Grid.append(nx.shortest_path_length(grid[i],temp[i][0],temp[i][1]))
    shortestPath_Random.append(nx.shortest_path_length(random[i],temp_rand[i][0],temp_rand[i][1]))
    onAmp_Grid.append(shortestPath_Grid[i]/5)
    onAmp_Random.append(shortestPath_Random[i]/5)

In [14]:
# seed[105]=np.random.randint(10000)

In [122]:
WS_Grid=[None]*len(avgDeg)
WS_Random=[None]*len(avgDeg)
for i in range(len(avgDeg)):
    WS_Grid[i]={'seed':[],'adj_matrix':[],'G':[],'Accuracy':{'Linear Transformation':[],'Mackey Glass':[],'MNIST':[]},'Graph Theory':{'Small World':[],'Modularity':[],'CCoeff':[],'MZ':[],'PCoeff':[],'PL':[]}}
    WS_Random[i]={'seed':[],'adj_matrix':[],'G':[],'Accuracy':{'Linear Transformation':[],'Mackey Glass':[],'MNIST':[]},'Graph Theory':{'Small World':[],'Modularity':[],'CCoeff':[],'MZ':[],'PCoeff':[],'PL':[]}}

In [123]:
# WS:
for i in tqdm(range(len(WS_Grid))):
    WS_Grid[i]['seed']=seed[i]
for i in tqdm(range(len(WS_Random))):
    WS_Random[i]['seed']=seed[i]
name='WS_Allnetworks_LinearTransformation.pkl'
print('Saving WS')
with open(name, 'wb') as f:
    pickle.dump([WS_Grid,WS_Random], f)   

Saving WS


In [79]:
def WS_Run(grid,random,onAmp_Grid,onAmp_Random,nodesList,WS_Grid,WS_Random,i):
    stimulus=[[],[]]
    stimulus_Rand=[[],[]]
    dt = 1e-2
    f=0.5
    Time=5
    stimulus[0]=(stimulus__(biasType='AC',onAmp=onAmp_Grid[i],TimeVector=np.arange(0,Time,dt),f=f))
    stimulus[1]=(stimulus__(biasType='Drain',TimeVector=np.arange(0,Time,dt))) #we don't want this drain to be active during training

    stimulus_Rand[0]=(stimulus__(biasType='AC',onAmp=onAmp_Random[i],TimeVector=np.arange(0,Time,dt),f=f))
    stimulus_Rand[1]=(stimulus__(biasType='Drain',TimeVector=np.arange(0,Time,dt))) #we don't want this drain to be active during training
    stimulus2 = [item for item in stimulus] #go through each list in the list and find the ith item
    stimulus2_Rand=[item for item in stimulus_Rand]

    results_grid=(runSim(connectivity__(graph=grid[i]),stimulus=stimulus2, contactMode='farthest', T = Time, dt = dt, onAmp = onAmp_Grid[i], biasType='AC',f=f,junctionMode='tunneling'))
    results_rand=(runSim(connectivity__(graph=random[i]),stimulus=stimulus2_Rand, contactMode='farthest', T = Time, dt = dt, onAmp = onAmp_Random[i], biasType='AC',f=f,junctionMode='tunneling'))
#     period=1/f
    
    nwSqu_Random=[None]*len(nodesList[1][i])
    nwSqu_Grid=[None]*len(nodesList[0][i])
    countk=0
    for k in nodesList[1][i]: #loop through sets of nodes for regression
        print('Running Random Network Regression: ' + str(k) + ' nodes')
        ResultSqu_Random=[]
        ResultSqu_Random = nonLinearTrans(results_rand,'Square',k, repeats=50) #simulation, type of signal, number of nodes to sample from, number of linear regression repetitions (take avg)
        nwSqu_Random[countk]=(ResultSqu_Random['accuracy'])
        countk=countk+1
        if k == nodesList[1][i][-1]: #if k is equal to the largest node sampling
            WS_Random[i]['Accuracy']['Linear Transformation']=[]
            WS_Random[i]['Accuracy']['Linear Transformation'].append(nwSqu_Random)
#             print(WS_Random[i]['Accuracy']['Linear Transformation'])
        else:
            WS_Random[i]['Accuracy']['Linear Transformation']=[None]
    countk=0
    for k in nodesList[0][i]:     
        print('Running Grid Network Regression: ' + str(k) + ' nodes')
        ResultSqu_Grid=[]
        ResultSqu_Grid = nonLinearTrans(results_grid,'Square',k, repeats=50) #simulation, type of signal, number of nodes to sample from, number of linear regression repetitions (take avg)
        nwSqu_Grid[countk]=(ResultSqu_Grid['accuracy'])
        countk=countk+1
        if k == nodesList[0][i][-1]: #if k is equal to the largest node sampling
            WS_Grid[i]['Accuracy']['Linear Transformation']=[]
            WS_Grid[i]['Accuracy']['Linear Transformation'].append(nwSqu_Grid)
#             print(WS_Grid[i]['Accuracy']['Linear Transformation'])
        else:
            WS_Grid[i]['Accuracy']['Linear Transformation']=[None]
    
    accGrid=nwSqu_Grid
    accRandom=nwSqu_Grid
    return accGrid,accRandom,WS_Grid,WS_Random

In [80]:
nodesList=[[None]*len(avgDeg) for i in range(2)]
for i in tqdm(range(len(avgDeg))):
    for j in range(2): #grid, random
        if j == 0:
            if len(grid[i]) >= 250:
                nodesList[j][i]=[50,100,150,200,250,len(grid[i])]#range(50, len(ws300[i][0])+1,50)
            elif len(grid[i]) >= 200 and len(grid[i]) < 250:
                nodesList[j][i]=[50,100,150,200,len(grid[i])]#range(50, len(ws300[i][0])+1,50)  
            elif len(grid[i]) >= 150 and len(grid[i]) < 200:
                nodesList[j][i]=[50,100,150,len(grid[i])]#range(50, len(ws300[i][0])+1,50)  
            elif len(grid[i]) >= 100 and len(grid[i]) < 150:
                nodesList[j][i]=[50,100,len(grid[i])]#range(50, len(ws300[i][0])+1,50)
        else:
            if len(random[i]) >= 250:
                nodesList[j][i]=[50,100,150,200,250,len(random[i])]#range(50, len(ws300[i][0])+1,50)
            elif len(random[i]) >= 200 and len(random[i]) < 250:
                nodesList[j][i]=[50,100,150,200,len(random[i])]#range(50, len(ws300[i][0])+1,50)  
            elif len(random[i]) >= 150 and len(random[i]) < 200:
                nodesList[j][i]=[50,100,150,len(random[i])]#range(50, len(ws300[i][0])+1,50)  
            elif len(random[i]) >= 100 and len(random[i]) < 150:
                nodesList[j][i]=[50,100,len(random[i])]#range(50, len(ws300[i][0])+1,50)

In [81]:
# results_grid=[]
# results_rand=[]
init=[]
for i in range(len(avgDeg)):
   init.append(inputPacker(WS_Run,grid,random,onAmp_Grid,onAmp_Random,nodesList,WS_Grid,WS_Random,i))

In [82]:
#parallel run simulations
result=para_run(WS_Run,init,nCPU=12)


2020-05-22 13:28:51,151:INFO:First current path [0, 290, 281, 272, 263, 254, 245, 236, 227, 218, 209, 200, 191, 182, 173, 164, 155, 146] formed at time = 0.75 s.


2020-05-22 13:28:52,672:INFO:First current path [0, 7, 21, 35, 49, 63, 77, 91, 105, 118, 131, 142] formed at time = 0.76 s.
2020-05-22 13:28:52,820:INFO:First current path [0, 23, 48, 16] formed at time = 0.75 s.


Running Random Network Regression: 50 nodes


Running Random Network Regression: 100 nodes
Running Random Network Regression: 150 nodes


2020-05-22 13:28:53,355:INFO:First current path [0, 281, 261, 241, 221, 201, 181, 161, 142] formed at time = 0.75 s.


...

Running Grid Network Regression: 50 nodes
Running Grid Network Regression: 100 nodes


2020-05-22 13:35:31,307:INFO:First current path [7, 73, 244, 208] formed at time = 0.75 s.


Running Random Network Regression: 50 nodes
Running Random Network Regression: 100 nodes
Running Grid Network Regression: 150 nodes
Running Grid Network Regression: 250 nodes
Running Random Network Regression: 150 nodes
Running Random Network Regression: 300 nodes
Running Grid Network Regression: 200 nodes
Running Random Network Regression: 200 nodes
Running Grid Network Regression: 300 nodes
Running Grid Network Regression: 250 nodes
Running Random Network Regression: 250 nodes


2020-05-22 13:35:32,653:INFO:First current path [5, 65, 206, 130] formed at time = 0.76 s.


Running Random Network Regression: 50 nodes
Running Grid Network Regression: 50 nodes
Running Random Network Regression: 100 nodes
Running Grid Network Regression: 100 nodes
Running Random Network Regression: 150 nodes
Running Grid Network Regression: 150 nodes
Running Grid Network Regression: 300 nodes
Running Random Network Regression: 300 nodes
Running Random Network Regression: 200 nodes
Running Grid Network Regression: 200 nodes
Running Random Network Regression: 250 nodes
Running Grid Network Regression: 250 nodes
Running Grid Network Regression: 50 nodes
Running Grid Network Regression: 100 nodes
Running Grid Network Regression: 150 nodes
Running Random Network Regression: 300 nodes
Running Grid Network Regression: 300 nodes
Running Grid Network Regression: 200 nodes
Running Grid Network Regression: 250 nodes
Running Grid Network Regression: 50 nodes
Running Grid Network Regression: 100 nodes
Running Grid Network Regression: 150 nodes
Running Grid Network Regression: 300 nodes
R

In [135]:
for i in range(len(result)):
    WS_Grid[i]['Accuracy']['Linear Transformation']=[ii[0] for ii in result][i]
    WS_Random[i]['Accuracy']['Linear Transformation']=[ii[1] for ii in result][i]

In [141]:
#export adj matrices to calculate small worldness in matlab:
adj_mats={"AdjMat_Grid":[],"AdjMat_Random":[]}
for i in tqdm(range(len(WS_Grid))):
    adj_mats['AdjMat_Grid'].append(dict(grid=nx.adj_matrix(grid[i]).todense()))
    adj_mats['AdjMat_Random'].append(dict(random=nx.adj_matrix(random[i]).todense()))
    WS_Grid[i]['adj_matrix']=nx.adj_matrix(grid[i]).todense()
    WS_Random[i]['adj_matrix']=nx.adj_matrix(random[i]).todense()
savemat('/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/300_WS_AllASN.mat',adj_mats)

In [143]:
cd "/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/"

/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity


In [144]:
smallworld_ws={'Grid':[],'Random':[]}
# temp=loadmat(r'300_WS_smallworld.mat')
temp=loadmat(r'300_WS_AllASN_smallworld.mat')
smallworld_ws['Grid']=temp['smallworld_grid']
smallworld_ws['Random']=temp['smallworld_random']
del temp

In [145]:
cd "/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks/"

/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks


In [ ]:
# WS:
ci = []
pcoeff= []
mz= []
clustering = []
for i in tqdm(range(len(WS_Grid))):
    ci,q=nct.community_louvain(np.squeeze(np.asarray(WS_Grid[i]['adj_matrix'])))
    pcoeff=bct.participation_coef(np.squeeze(np.asarray(WS_Grid[i]['adj_matrix'])),ci)
    mz=bct.module_degree_zscore(np.squeeze(np.asarray(WS_Grid[i]['adj_matrix'])),ci)
    WS_Grid[i]['G']=grid[i]
    clustering=nx.clustering(WS_Grid[i]['G'])
    WS_Grid[i]['Graph Theory']['PL']=dict(nx.all_pairs_shortest_path_length(WS_Grid[i]['G']))
    WS_Grid[i]['Graph Theory']['Modularity']=ci
    WS_Grid[i]['Graph Theory']['Modularity Score']=q
    WS_Grid[i]['Graph Theory']['PCoeff']=pcoeff
    WS_Grid[i]['Graph Theory']['MZ']=mz
    WS_Grid[i]['Graph Theory']['Small World']=smallworld_ws['Grid'][0][i]
    WS_Grid[i]['Graph Theory']['CCoeff']=clustering
    WS_Grid[i]['Graph Theory']['Degree']=nx.degree(WS_Grid[i]['G'])
    WS_Grid[i]['Graph Theory']['AvgPL']=nx.average_shortest_path_length(WS_Grid[i]['G'])

ci = []
pcoeff= []
mz= []
clustering = []
count1=0
count2=0
count3=0   
for i in tqdm(range(len(WS_Random))):
    ci,q=nct.community_louvain(np.squeeze(np.asarray(WS_Random[i]['adj_matrix'])))
    pcoeff=bct.participation_coef(np.squeeze(np.asarray(WS_Random[i]['adj_matrix'])),ci)
    mz=bct.module_degree_zscore(np.squeeze(np.asarray(WS_Random[i]['adj_matrix'])),ci)
    WS_Random[i]['G']=random[i]
    clustering=nx.clustering(WS_Random[i]['G'])
    WS_Random[i]['Graph Theory']['PL']=dict(nx.all_pairs_shortest_path_length(WS_Random[i]['G']))
    WS_Random[i]['Graph Theory']['Modularity']=ci
    WS_Random[i]['Graph Theory']['Modularity Score']=q
    WS_Random[i]['Graph Theory']['PCoeff']=pcoeff
    WS_Random[i]['Graph Theory']['MZ']=mz
    WS_Random[i]['Graph Theory']['Small World']=smallworld_ws['Random'][0][i]
    WS_Random[i]['Graph Theory']['CCoeff']=clustering
    WS_Random[i]['Graph Theory']['Degree']=nx.degree(WS_Random[i]['G'])
    WS_Random[i]['Graph Theory']['AvgPL']=nx.average_shortest_path_length(WS_Random[i]['G'])    

name='WS_Allnetworks_LinearTransformation.pkl'
print('Saving WS')
with open(name, 'wb') as f:
    pickle.dump([WS_Grid,WS_Random], f)   

In [248]:
for i in range(len(WS_Grid)):
    print(np.max(WS_Grid[i]['Accuracy']['Linear Transformation']))

0.6845246805652542
0.7255613395828878
0.7468681171415351
0.7571770365915949
0.7892507844001515
0.7845780416845965
0.8034356645410518
0.8131583984307248
0.8073213176754855
0.7950913465755682
0.6254715264795581
0.7149951718888772
0.72556133958294
0.7762802766417038
0.7571770365915884
0.7461231878900189
0.824315255225879
0.812724150489894
0.7863053972221798
0.8185311149250077
0.6054637523204425
0.6568946339601802
0.7149951718929696
0.7255613395829186
0.7762802766417063
0.7571770365915859
0.7461231878900334
0.7892507844001533
0.7946915793394408
0.803435664541062
0.6798640951246585
0.6477455123543239
0.6860579929155223
0.7726359518011595
0.7031423433752085
0.7468681171415041
0.7571770365915883
0.7461231878900179
0.7845128206299238
0.8243152552258285
0.6773166759267941
0.6200479918823733
0.6477455117359228
0.7149951718901368
0.7508479039362033
0.7018715068603341
0.7762802766417143
0.7571770365915806
0.7453659384252899
0.7461231878900335
0.5670507109225514
0.6798640946491773
0.625471526482693

In [ ]:
WS_Grid[0]